# Total work effort in Denmark compared to other OECD countries

Imports and set magics:

In [ ]:
#pip install pandasdmx
#pip install geopandas

In [ ]:
import pandas as pd
import pandasdmx as pdmx
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import geopandas as gpd
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject

# Read and clean data

In this data project we will investigate the total work effort in Denmark. In relation to this we investigate how the working hours per person employed in OECD countries have evolved over time and especially how Denmark performs compared to the other OECD countries. Additionaly, we explore the relationship between employment rates and working hours per person employed across OECD countries. 

The two used datasets for the analysis is imported from the OECD database (https://data.oecd.org). Both datasets contain annual data from 2008 to 2022. We employ the following datasets:
-  **Level of GDP per capita and productivity - Average hours worked per person employed:** The dataset contains data on average hours worked per person employed measured as hours per year in OECD countries. 
- **Short-Term Labour Market Statistics - Employment Rate:** The dataset contains data on employment rates measured as the percentage of 15-64 year olds who are employed in each OECD country. The dataset contains employment rates for females, males and all. 

Importing data on employment rates:

In [ ]:
# Fetching data
emplrate = dataproject.fetching_data_emplrate()
emplrate.head()

Cleaning the data:

In [ ]:
# Drop measure and frequency
emplrate.drop(['MEASURE', 'FREQUENCY'], axis=1, inplace=True) 

# Rename 
emplrate.rename(columns = {'TIME_PERIOD':'YEAR', 'value':'EMPLRATE'}, inplace=True)
emplrate['SUBJECT'].replace({'LREM64FE': 'Female', 'LREM64MA': 'Male', 'LREM64TT': 'All'}, inplace=True)
emplrate

Importing data on average hours worked per person employed:

In [ ]:
# Fetching data
hours = dataproject.fetching_data_hours()
hours.head()

Cleaning the data:

In [ ]:
# Drop subject and measure
hours.drop(['SUBJECT', 'MEASURE'], axis=1, inplace=True) 

# Rename 
hours.rename(columns = {'TIME_PERIOD':'YEAR', 'value':'AVHRS'}, inplace=True)
hours

## Explore each data set

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

**Vi mangler mere deskriptiv statistik og det kunne være nice at lave noget med det der split-and-apply - fx nogle genenmsnit for alle EU lande eller på tværs af køn - I dont know.** 

An interactive plot illustrates the employments rates of 2022 across OECD countries with a drop down menu that enables the graph to show employment rates for males, females and all, respectively. This shows which countries are included in the 'emplrate' dataset and indicates the differences in employment rates across OECD countries:

In [ ]:
# Import world map data
worldmap = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

def create_map_emplrate(gender):
    # Looking at employment rates in 2022
    emplrate_map = emplrate[(emplrate['SUBJECT'] == gender) & (emplrate['YEAR'] == '2022')]

    # Merge employment rate data and world map data
    mapdata_emplrate = pd.merge(worldmap, emplrate_map, how="left", left_on='iso_a3', right_on='LOCATION')

    # Plot the map
    fig, ax = plt.subplots(1, 1, figsize=(20, 16))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2%", pad="0.5%")
    mapdata_emplrate.plot(column="EMPLRATE", ax=ax, cax=cax, cmap='OrRd', legend=True, legend_kwds={"label": f"Employment rate in 2022 for {gender} (%)"}, missing_kwds={'color':'lightgrey'}, vmin=45, vmax=85)
    ax.set_title(f'Employment rate for OECD countries in 2022', size=20)
    plt.show()

# Dropdown menu 
gender_dropdown = widgets.Dropdown(options=['All', 'Male', 'Female'], value='All', description='Gender:')
widgets.interactive(create_map_emplrate, gender=gender_dropdown)

Another world map illustrates the average hours worked per person employed in 2022 across OECD countries. This shows which countries are included in the 'hours' dataset and indicates the differences in working hours across OECD countries:

In [ ]:
# Looking at working hours in 2022
hours_map = hours[hours['YEAR'] == '2022']

# Merge employment rate data and world map data
mapdata_hours = pd.merge(worldmap, hours_map, how="left", left_on='iso_a3', right_on='LOCATION')

# Plot the map
fig, ax = plt.subplots(1, 1, figsize=(20, 16))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="2%", pad="0.5%")
mapdata_hours.plot(column="AVHRS", ax=ax, cax=cax, cmap='OrRd', legend=True, legend_kwds={"label": f"Average working hours per person employed (hours per year)"}, missing_kwds={'color':'lightgrey'}, vmin=1400, vmax=2000)
ax.set_title(f'Average working hours per person employed in OECD countries in 2022', size=20)
plt.show()

# Merge data sets

Merge the datasets on employment rates and average hours per person employed in OECD countries. We do an inner merge because we only want to keep observations where both an employment rate and average working hours exists:

In [ ]:
#Merge the two dataset
data_merged = pd.merge(emplrate,hours,how='inner',on=['LOCATION','YEAR'])
data_merged

# Analysis

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

We plot the average hours worked per person employed over time for each of the OECD countries in the analysis to get an idea of how the working hours have evolved across time:

In [ ]:
# Define function to update plot based on selected LOCATION
def update_plot(location):
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(1,1,1)
    ax.plot(data_merged[(data_merged['LOCATION'] == location) & (data_merged['SUBJECT'] == 'All')]['YEAR'], 
            data_merged[(data_merged['LOCATION'] == location) & (data_merged['SUBJECT'] == 'All')]['AVHRS'])
    ax.set_xlabel('Year')
    ax.set_ylabel('Average hours worked per person employed (hours per year)')
    ax.set_title(f'Average hours worked per person employed across time for {location}')
    plt.show()

# Create dropdown menu for selecting LOCATION
location_dropdown = widgets.Dropdown(options=data_merged['LOCATION'].unique(), value='DNK', description='Location:')

# Define interactive widget to update plot based on dropdown selection
widgets.interactive(update_plot, location=location_dropdown)

There is a general trend in OECD countries where working hours per person employed have declined in the period 2008-2022. 

In a scatterplot the employments rate is plotted against the average hours worked in 2022 for all OECD countries:

In [ ]:
# Plot employment rates across average working hours per person employed
plt.figure(figsize=(10, 6))
plt.scatter(x=data_merged[(data_merged['SUBJECT'] == 'All') & (data_merged['YEAR'] == '2022')]['AVHRS'], 
            y=data_merged[(data_merged['SUBJECT'] == 'All') & (data_merged['YEAR'] == '2022')]['EMPLRATE'])

plt.xlabel('Average hours worked per person employed (hours per year)')
plt.ylabel('Employment rate (%)')
plt.title('Scatter plot of employment rate against average working hours across OECD countries (2022)')

# Annotate points with country names
for index, row in data_merged[(data_merged['SUBJECT'] == 'All') & (data_merged['YEAR'] == '2022')].iterrows():
    plt.annotate(row['LOCATION'], (row['AVHRS'], row['EMPLRATE']))

plt.show()

The graph shows that Denmark ranks very low when it comes to working hours per person employed. At the same time relatively many danes between 15 and 64 years is in employment compared to other OECD countries.

A bar chart compares the employment rates in 2022 of OECD countries and highlights the rank of Denmark. A drop down menu enables the plot to how employment rates for males and females:

In [ ]:
# Plotting a bar chart of the male and female employment rates in 2022
def update_plot(gender):
    plt.figure(figsize=(10, 6))
    
    # Filter data for the selected gender and year
    filtered_data = data_merged[(data_merged['YEAR'] == '2022') & (data_merged['SUBJECT'] == gender)]

    # Sort the data in descending order of employment rate
    sorted_data = filtered_data.sort_values(by='EMPLRATE', ascending=False)

     # Plotting the data
    plt.bar(sorted_data['LOCATION'], sorted_data['EMPLRATE'], color='skyblue')
    plt.bar(sorted_data[sorted_data['LOCATION'] == 'DNK']['LOCATION'], sorted_data[sorted_data['LOCATION'] == 'DNK']['EMPLRATE'], color='blue')
    
    plt.xlabel('Country')
    plt.ylabel('Employment Rate (%)')
    plt.title(f'{gender} Employment Rates in OECD Countries in 2022')
    plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
    plt.show()

# Create dropdown menu for selecting gender
gender_dropdown = widgets.Dropdown(options=['Male', 'Female'], value='Female', description='Gender:')

# Define interactive widget to update plot based on dropdown selection
widgets.interactive(update_plot, gender=gender_dropdown)

The female employment rate is especially large in Denmark compared to the other OECD countries. 

# Conclusion

Our analysis shows that average working hours per person employed has decreased across time for most of the OECD countries. 
Specifically, Denmark ranks as one of the OECD countries with the lowest average working hours per person employed in 2022.However, the employment rate in Denmark in 2022 is quite large compared to a lot of the other OECD countries. This is, among other things, because the female employment rate in Denmark is relatively large. Thus we cannot conclude anything umambiguous about the total work effort in Denmark. 